# Adversarial example using pytorch

In this example, we redo the [adversarial example](https://gurobi-optimization-ml2gurobi.readthedocs-hosted.com/en/latest/examples/adversarial_mnist.html) of the documentation but use pytorch for training the neural network.

We don't detail the optimization model here. Please refer to the example in the documentation.

This example requires the additional packages:
 - [torch](https://pytorch.org/)
 - [torchvision](https://pytorch.org/)
 - [matplotlib](https://matplotlib.org/)
 - [skorch](https://skorch.readthedocs.io/en/stable/)

 The latter package is a wrapper for giving to pytorch a Scikit-Learn like interface.

## Import the necessary packages and load data

We import all the package we need for this example.
We fetch the MINST data set using sklearn's functionalities.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import torch
import torchvision
from skorch import NeuralNetClassifier

import gurobipy as gp

from gurobi_ml import add_predictor_constr

In [ ]:
# Get MNIST digit recognition data set
mnist_train = torchvision.datasets.MNIST(root="./MNIST", train=True, download=True)

mnist_test = torchvision.datasets.MNIST(root="./MNIST", train=False, download=True)

Flatten and scale the data

In [ ]:
x_train = torch.flatten(mnist_train.data.type(torch.FloatTensor), start_dim=1)
y_train = mnist_train.targets
x_test = torch.flatten(mnist_test.data.type(torch.FloatTensor), start_dim=1)
y_test = mnist_test.targets

x_train /= 255.0  # scaling
x_test /= 255.0  # scaling

## Construct and train the neural network

We construct a sequential neural network with 2 hidden layers of 50 neurons.
To train it, we use `skorch` that provides an interface similar to `scikit-learn`.

In [ ]:
nn_model = torch.nn.Sequential(
    torch.nn.Linear(28 * 28, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 10),
    torch.nn.Softmax(1),
)

In [ ]:
clf = NeuralNetClassifier(
    nn_model,
    max_epochs=50,
    lr=0.1,
    iterator_train__shuffle=True,
)

clf.fit(X=x_train, y=y_train)

In [ ]:
print(f"Training score: {clf.score(x_train, y_train):.4}")
print(f"Validation set score: {clf.score(x_test, y_test):.4}")

In [ ]:
nn_regression = torch.nn.Sequential(*[layer for layer in nn_model[:-1]])

In [ ]:
imageno = 10000
image = mnist_train.data[imageno, :]
plt.imshow(image, cmap="gray")

In [ ]:
ex_prob = nn_regression.forward(x_train[imageno, :])
sorted_labels = torch.argsort(ex_prob)
right_label = sorted_labels[-1]
wrong_label = sorted_labels[-2]

In [ ]:
image = x_train[imageno, :].numpy()  # We need numpy converted image

m = gp.Model()
delta = 5

x = m.addMVar(image.shape, lb=0.0, ub=1.0, name="x")
y = m.addMVar(ex_prob.detach().numpy().shape, lb=-gp.GRB.INFINITY, name="y")

abs_diff = m.addMVar(image.shape, lb=0, ub=1, name="abs_diff")

m.setObjective(y[wrong_label] - y[right_label], gp.GRB.MAXIMIZE)

# Bound on the distance to example in norm-1
m.addConstr(abs_diff >= x - image)
m.addConstr(abs_diff >= -x + image)
m.addConstr(abs_diff.sum() <= delta)

pred_constr = add_predictor_constr(m, nn_regression, x, y)

pred_constr.print_stats()

In [ ]:
m.Params.BestBdStop = 0.0
m.Params.BestObjStop = 0.0
m.optimize()

In [ ]:
if m.ObjVal > 0.0:
    plt.imshow(x.X.reshape((28, 28)), cmap="gray")
    label = tf.math.argmax(nn.predict(tf.reshape(x.X, (1, -1))), axis=1)
    print(f"Solution is classified as {label}")
else:
    print("No counter example exists in neighborhood.")

copyright © 2022 Gurobi Optimization, LLC